In [35]:
import pandas as pd
import os

In [43]:
sumZoneTripByMode =  'data/tripsByMode'
scnJson = '_site/scenarios.json'
outputJsonName = 'zones-modetrips-vizmap.json'
fnSuffix = '_ZoneSummary_TripsByMode.csv'
scenarioDataFolder = '_site/data/scnData'

In [37]:
# attributes for road seg summaries
dfAttributes = pd.DataFrame([
    ['aAll' ,'All Trips'                       ,'All' ],
    ['aMoto','Motorized Trips'                 ,'Moto'],
    ['aAuto','Auto Trips'                      ,'Auto'],
    ['atT'  ,'All Transit Trips'               ,'tTrn'],
    ['atl'  ,'Local Bus Trips'                 ,'tLCL'],
    ['atC'  ,'Core Bus Trips'                  ,'tCOR'],
    ['atE'  ,'Express Bus Trips'               ,'tEXP'],
    ['atL'  ,'Light Rail Trips'                ,'tLRT'],
    ['atC'  ,'Commuter Rail Trips'             ,'tCRT'],
    ['atB'  ,'Bus Rapid Transit Trips'         ,'tBRT'],
    ['adT'  ,'Drive-to-All Transit Trips'      ,'dTrn'],
    ['adl'  ,'Drive-to-Local Bus Trips'        ,'dLCL'],
    ['adC'  ,'Drive-to-Core Bus Trips'         ,'dCOR'],
    ['adE'  ,'Drive-to-Express Bus Trips'      ,'dEXP'],
    ['adL'  ,'Drive-to-Light Rail Trips'       ,'dLRT'],
    ['adC'  ,'Drive-to-Commuter Rail Trips'    ,'dCRT'],
    ['adB'  ,'Dirve-to-Bus Rapid Transit Trips','dBRT'],
    ['awT'  ,'Walk-to-All Transit Trips'       ,'wTrn'],
    ['awl'  ,'Walk-to-Local Bus Trips'         ,'wLCL'],
    ['awC'  ,'Walk-to-Core Bus Trips'          ,'wCOR'],
    ['awE'  ,'Walk-to-Express Bus Trips'       ,'wEXP'],
    ['awL'  ,'Walk-to-Light Rail Trips'        ,'wLRT'],
    ['awC'  ,'Walk-to-Commuter Rail Trips'     ,'wCRT'],
    ['awB'  ,'Walk-to-Bus Rapid Transit Trips' ,'wBRT'],
    ['aNM'  ,'Non-Motorized Trips'             ,'NonM'],
    ['aWk'  ,'Walk Trips'                      ,'Walk'],
    ['aBk'  ,'Bike Trips'                      ,'Bike']
], columns=('aCode','name','colName'))
display(dfAttributes)

dfFilters = pd.DataFrame([
    ["fPurp","Trip Purpose"         , "select", True ],
    ["fTod" ,"Time of Day"          , "select", True ],
    ["fPA"  ,"Production/Attraction", "select", True ]
], columns=('fCode','fName','fWidget','fOwnCol'))
display(dfFilters)

lstFiltersOwnCol = dfFilters[dfFilters['fOwnCol'] == True]['fCode'].tolist()
display(lstFiltersOwnCol)


,aCode,name,colName
0,aAll,All Trips,All
1,aMoto,Motorized Trips,Moto
2,aAuto,Auto Trips,Auto
3,atT,All Transit Trips,tTrn
4,atl,Local Bus Trips,tLCL
5,atC,Core Bus Trips,tCOR
6,atE,Express Bus Trips,tEXP
7,atL,Light Rail Trips,tLRT
8,atC,Commuter Rail Trips,tCRT
9,atB,Bus Rapid Transit Trips,tBRT


,fCode,fName,fWidget,fOwnCol
0,fPurp,Trip Purpose,select,True
1,fTod,Time of Day,select,True
2,fPA,Production/Attraction,select,True


['fPurp', 'fTod', 'fPA']

In [38]:
dfScn = pd.read_json(scnJson)
dfScn = dfScn[['modVersion','scnGroup','scnYear']]
display(dfScn)
display(dfScn.dtypes)

,modVersion,scnGroup,scnYear
0,v900,Base,2019
1,v900,Base,2023
2,v900,TIP,2028
3,v900,RTP,2032
4,v900,RTP,2042
5,v900,RTP,2050
6,v900,NoBuild,2032
7,v900,NoBuild,2042
8,v900,NoBuild,2050
9,v900,Needs,2032


modVersion    object
scnGroup      object
scnYear        int64
dtype: object

In [39]:
# create folders
# Define a root directory where you want to create subfolders
root_dir = './_site/data/scnData'

# Ensure the root directory exists
if not os.path.exists(root_dir):
    os.makedirs(root_dir)

# Loop through the dataframe and create subfolders
for index, row in dfScn.iterrows():
    folder_name = f"{row['modVersion']}__{row['scnGroup']}__{row['scnYear']}"
    full_path = os.path.join(root_dir, folder_name)
    
    if not os.path.exists(full_path):
        os.makedirs(full_path)


In [40]:
import pandas as pd
import os
import glob

# Create an empty list to store dataframes
_dfs = []

# Iterate through the scenarios
for _, row in dfScn.iterrows():
    print(row['scnGroup'] + '_' + str(row['scnYear']))
    
    # Read the CSV into a dataframe
    _df = pd.read_csv(sumZoneTripByMode + '/' + row['scnGroup'] + '_' + str(row['scnYear']) + fnSuffix)
    #display(_df)

    _df_melt = _df.melt(id_vars=['TAZID'] + lstFiltersOwnCol, value_vars=dfAttributes['colName'].tolist(), var_name='colName', value_name='val')
    #display(_df_melt)
    _df_melt_withkeys = pd.DataFrame.merge(_df_melt, dfAttributes, on='colName')
    #display(_df_melt_withkeys)

    _df_melt_withkeys_pivot_atts = _df_melt_withkeys.pivot(index=[col for col in _df_melt_withkeys.columns if col not in ['aCode','val','colName']], columns='aCode', values="val")
    _df_melt_withkeys_pivot_atts.columns.name = None
    _df_melt_withkeys_pivot_atts.reset_index(inplace=True)
    _df_melt_withkeys_pivot_atts
    #display(_df_melt_withkeys_pivot_atts)

    # Add new columns with values from filename
    _df_melt_withkeys_pivot_atts['modVersion'] = row['modVersion']
    _df_melt_withkeys_pivot_atts['scnGroup'] = row['scnGroup']
    _df_melt_withkeys_pivot_atts['scnYear'] = row['scnYear']

    _df_melt_withkeys_pivot_atts['scnYear'] = _df_melt_withkeys_pivot_atts['scnYear'].astype('int64')
    #display(_df_melt_withkeys_pivot_atts)

    _dfs.append(_df_melt_withkeys_pivot_atts)

# If you want to concatenate all the dataframes into a single dataframe
print('combining dataframes...')
dfCombined_melt_withkeys_pivot_atts = pd.concat(_dfs, ignore_index=True)
display(dfCombined_melt_withkeys_pivot_atts)
print('Done!')


Base_2019
Base_2023
TIP_2028
RTP_2032
RTP_2042
RTP_2050
NoBuild_2032
NoBuild_2042
NoBuild_2050
Needs_2032
Needs_2042
Needs_2050
Needs MAG_2050
combining dataframes...


,TAZID,fPurp,fTod,fPA,name,aAll,aAuto,aBk,aMoto,aNM,...,atl,awB,awC,awE,awL,awT,awl,modVersion,scnGroup,scnYear
0,2,All,Dy,a,All Transit Trips,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,v900,Base,2019
1,2,All,Dy,a,All Trips,38.8419,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,v900,Base,2019
2,2,All,Dy,a,Auto Trips,NaN,33.0394,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,v900,Base,2019
3,2,All,Dy,a,Bike Trips,NaN,NaN,0.3943,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,v900,Base,2019
4,2,All,Dy,a,Bus Rapid Transit Trips,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,v900,Base,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38816005,3629,HBW,Pk,p,Walk-to-Commuter Rail Trips,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,v900,Needs MAG,2050
38816006,3629,HBW,Pk,p,Walk-to-Core Bus Trips,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,v900,Needs MAG,2050
38816007,3629,HBW,Pk,p,Walk-to-Express Bus Trips,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,v900,Needs MAG,2050
38816008,3629,HBW,Pk,p,Walk-to-Light Rail Trips,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,v900,Needs MAG,2050


Done!


In [41]:
## replace zeros with NaN
#import numpy as np
#dfCombined_melt_withkeys_pivot_atts.replace(0, np.nan, inplace=True)
#display(dfCombined_melt_withkeys_pivot_atts)

In [42]:
import json

# PREPARE JSON FOR EACH SCENARIO
# scenarioDetails
# attributes
#   aCode
#   aName
#   filterGroup
# filterGroups
#   possible filterGroup Options
#   for each filterGroup possibility, the data for that selection

_df0 = dfCombined_melt_withkeys_pivot_atts.copy()

# Loop through scenarios in dfScn
for _, row in dfScn.iterrows():

    _scenarioCode = row['modVersion'] + '__' + row['scnGroup'] + '__' + str(row['scnYear'])
    print(_scenarioCode)

    # format scenario details for json export
    jsonScenarioDetails = row.to_dict()

    #print(row['modVersion'] + row['scnGroup'] + str(row['scnYear']))
    # Filter the main dataframe based on the scenario
    _df0_filtered = _df0[
        (_df0['modVersion'] == row['modVersion']) & 
        (_df0['scnGroup'] == row['scnGroup']) & 
        (_df0['scnYear'] == row['scnYear'])
    ].copy()

    _df = _df0_filtered.copy()
    #display(_df)

    # add options to _dfFiltersWithOptions
    
    _dfFiltersWithOptions = dfFilters.copy()

    # For each fCode, get the unique values from dfData, and join to dfFilters
    formatted_lists = []
    for fCode in dfFilters['fCode']:
        unique_values = [x for x in _df[fCode].unique() if pd.notna(x)]
        formatted_lists.append(unique_values)
    #display (formatted_lists)

    _dfFiltersWithOptions['fOptions'] = formatted_lists    
    #display(_dfFiltersWithOptions)

    # get list of fitler groups (any column that begins with f)
    fCols = [col for col in dfCombined_melt_withkeys_pivot_atts.columns if col.startswith('f')]
    _df[fCols] = _df[fCols].fillna('')
    _dfFilters = _df[fCols].drop_duplicates()

    # Drop rows with empty strings in ALL columns
    #_dfFilters = _dfFilters[~(_dfFilters == '').all(axis=1)]
    #display(_dfFilters)

    # Convert the values to binary form, so 1 where there is a value and 0 elsewhere
    binary_dfFilters = _dfFilters.applymap(lambda x: 0 if x=='' else 1)
    #display(binary_dfFilters)

    # Get unique rows of combinations
    unique_combinations = binary_dfFilters.drop_duplicates()

    # Convert the unique combinations to a list of tuples
    lstCombinations = list(unique_combinations.itertuples(index=False, name=None))
    #display(lstCombinations)

    # Define a function to convert to binary
    def is_empty_or_not(value):
        return 0 if value=='' else 1

    _dfFilterACodes = pd.DataFrame()

    jsonDataCombined = {}

    # loop through all combinations in list
    for combination in lstCombinations:

        # get list of column names with a 1 for given combination
        _dfFilterColsWithValue1 = [col for idx, col in enumerate(fCols) if combination[idx] == 1]
        #display(_dfFilterColsWithValue1)

        # create string of column names
        _strFilterGroup = '_'.join(_dfFilterColsWithValue1)
        #display(_strFilterGroup)
        
        # Initialize the condition as True for all rows
        condition = pd.Series([True] * len(_df), index=_df.index)

        # Apply the condition programmatically for each column in the list
        for col, comb_value in zip(fCols, combination):
            condition &= _df[col].apply(is_empty_or_not) == comb_value

        _filtered_df = _df[condition].copy()
        _filtered_df.drop(columns=['modVersion','scnGroup','scnYear'], inplace=True)
        #display(_filtered_df)

        # Remove columns where all values are NaN
        _filtered_df_noNA = _filtered_df.dropna(axis=1, how='all')
        #display(_filtered_df_noNA)

        _fColsToDrop = [col for col in _filtered_df_noNA.columns if col.startswith('f') and (_filtered_df_noNA[col] == '').all()]

        # Drop the identified columns - only have fCols with values
        _df2 = _filtered_df_noNA.drop(columns=_fColsToDrop)
        #display(_df2)

        # Get list of columns that begin with "a", create a dataframe the filter group for all possible combinations
        aCols = [col for col in _filtered_df_noNA.columns if col.startswith('a')]
        new_data = pd.DataFrame([{"filterGroup": _strFilterGroup, 'aCode': aCols}])
        new_data = new_data.explode('aCode').reset_index(drop=True)
        _dfFilterACodes = pd.concat([_dfFilterACodes, new_data], ignore_index=True)

        # get unique filter values
        _fColsRemain = [col for col in _df2.columns if col.startswith('f')]
        #display(_fColsRemain)
        if _fColsRemain:
            _dfFilterOptions = _df2[_fColsRemain].drop_duplicates()
        else:
            # add dummy row for no filter
            _dfFilterOptions = pd.DataFrame({'nofilter': ['']})

        #display(_dfFilterOptions)

        # Handle no filter data

        first_row = True

        for index, row in _dfFilterOptions.iterrows():

            _filterRowIndex = '_'.join(row.astype(str).values)

            print(_filterRowIndex)
            
            final_condition = ""

            if _fColsRemain != []:
                # Dynamically construct the filter condition
                shared_columns = set(_dfFilterOptions.columns) & set(_df2.columns)
                conditions = (_df2[col].isin([row[col]]) for col in shared_columns)
                final_condition = next(conditions)

                for condition in conditions:
                    final_condition &= condition

                filtered_df2 = _df2[final_condition].copy()
                filtered_df2.drop(columns=_fColsRemain, inplace=True)
            else: # no filter condition
                filtered_df2 =  _df2

            _df2noI = filtered_df2.reset_index()

            jsonDataForFilterOption = {}

            for index, row in _df2noI.iterrows():
                # Initialize an empty dictionary for the row
                row_dict = {}

                # Loop through each column that starts with 'a'
                for col in _df2noI.columns[_df2noI.columns.str.startswith('a')]:
                    # Add to row_dict if value is greater than 0
                    if row[col] > 0:
                        row_dict[col] = row[col]
                
                # If row_dict is not empty, add it to jsonDataCombined with TAZID as the key
                if row_dict:
                    jsonDataForFilterOption[int(row['TAZID'])] = row_dict  # Cast TAZID to int
            
            # Encapsulate jsonDataCombined under the _filterRowIndex
            final_json = {_filterRowIndex: jsonDataForFilterOption}

            ## Convert to a JSON formatted string
            #json_output = json.dumps(final_json)

            # Append the JSON structure from this iteration to the combined
            jsonDataCombined = {**jsonDataCombined, **final_json}

    _dfAttributesWFilter = pd.DataFrame.merge(dfAttributes, _dfFilterACodes, on='aCode')
    jsonAttributes = _dfAttributesWFilter.to_dict(orient='records')
    jsonFilters = _dfFiltersWithOptions.to_dict(orient='records')

    # Construct final JSON structure
    jsonScenario = {
        'scenarioDetails': jsonScenarioDetails,
        'attributes': jsonAttributes,
        'filters': jsonFilters,
        'data': jsonDataCombined#,
        #'aggregations': aggregations_list
    }

    # Specify the desired file name
    filename = scenarioDataFolder '/' + _scenarioCode + '/' + outputJsonName

    # Open the file in write mode
    with open(filename, 'w') as file:
        json.dump(jsonScenario, file)

v900__Base__2019
All_Dy_a
All_Dy_p
All_Ok_a
All_Ok_p
All_Pk_a
All_Pk_p
HBO_Dy_a
HBO_Dy_p
HBO_Ok_a
HBO_Ok_p
HBO_Pk_a
HBO_Pk_p
HBSch_Dy_a
HBSch_Dy_p
HBSch_Ok_a
HBSch_Ok_p
HBSch_Pk_a
HBSch_Pk_p
HBW_Dy_a
HBW_Dy_p
HBW_Ok_a
HBW_Ok_p
HBW_Pk_a
HBW_Pk_p
NHB_Dy_a
NHB_Dy_p
NHB_Ok_a
NHB_Ok_p
NHB_Pk_a
NHB_Pk_p
HBC_Dy_a
HBC_Dy_p
HBC_Pk_a
HBC_Pk_p
v900__Base__2023
All_Dy_a
All_Dy_p
All_Ok_a
All_Ok_p
All_Pk_a
All_Pk_p
HBO_Dy_a
HBO_Dy_p
HBO_Ok_a
HBO_Ok_p
HBO_Pk_a
HBO_Pk_p
HBSch_Dy_a
HBSch_Dy_p
HBSch_Ok_a
HBSch_Ok_p
HBSch_Pk_a
HBSch_Pk_p
HBW_Dy_a
HBW_Dy_p
HBW_Ok_a
HBW_Ok_p
HBW_Pk_a
HBW_Pk_p
NHB_Dy_a
NHB_Dy_p
NHB_Ok_a
NHB_Ok_p
NHB_Pk_a
NHB_Pk_p
HBC_Dy_a
HBC_Dy_p
HBC_Pk_a
HBC_Pk_p
v900__TIP__2028
All_Dy_a
All_Dy_p
All_Ok_a
All_Ok_p
All_Pk_a
All_Pk_p
HBO_Dy_a
HBO_Dy_p
HBO_Ok_a
HBO_Ok_p
HBO_Pk_a
HBO_Pk_p
HBSch_Dy_a
HBSch_Dy_p
HBSch_Ok_a
HBSch_Ok_p
HBSch_Pk_a
HBSch_Pk_p
HBW_Dy_a
HBW_Dy_p
HBW_Ok_a
HBW_Ok_p
HBW_Pk_a
HBW_Pk_p
NHB_Dy_a
NHB_Dy_p
NHB_Ok_a
NHB_Ok_p
NHB_Pk_a
NHB_Pk_p
HBC_Dy_a
HBC_Dy_p
HBC_Pk_a
HBC_P